In [1]:
import cv2
from data_sensor import get_img_coord
import angle_out as angle

import yaml
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import pickle
import json

with open("Zoo/config.yaml") as f:
    conf_yaml = yaml.load(f, Loader=yaml.FullLoader)

    body_parts = []
    link_parts = []
    for part in conf_yaml["bodyparts"]:
        body_parts.append(part)
    for link in conf_yaml["skeleton"]:
        link_parts.append(link)


label_parts = [
    ["nose", "neck_base", "neck_end"],
    ["neck_base", "neck_end", "back_base"],
    ["neck_end", "back_base", "back_middle"],
    ["back_base", "back_middle", "back_end"],
    ["back_middle", "back_end", "tail_base"],
    ["back_end", "tail_base", "tail_end"],
    ["back_base", "front_left_thai", "front_left_knee"],
    ["front_left_thai", "front_left_knee", "front_left_paw"],
    ["back_base", "front_right_thai", "front_right_knee"],
    ["front_right_thai", "front_right_knee", "front_right_paw"],
    ["back_end", "back_left_thai", "back_left_knee"],
    ["back_left_thai", "back_left_knee", "back_left_paw"],
    ["back_end", "back_right_thai", "back_right_knee"],
    ["back_right_thai", "back_right_knee", "back_right_paw"],
]

with open("model/xgboost.pkl", "rb") as f:
    model = pickle.load(f)

with open("model/encoder.pkl", "rb") as f:
    encoder = pickle.load(f)
with open("model/label.json", "rb") as f:
    json_ = json.load(f)


2023-08-11 16:58:58.962425: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading DLC 2.3.5...


/home/pieroot/miniconda3/envs/dlc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pieroot/miniconda3/envs/dlc/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [2]:
def predict(video_save=False):
    video = "videos/HEADING_video/dog-heading-005831.mp4"
    cap = cv2.VideoCapture(video)

    # 영상 기본 정보
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 영상의 가공 정보
    window_size = 10
    flag = int(int(fps) / (30 / 5))
    count = 1

    if cap.isOpened():
        ret, frame = cap.read()
        print(f"width: {width}, height: {height}, fps: {fps}")
        label = []
        result = ""
        out_video = []
        width_tmp = 720
        height_tmp = 480
        while ret:
            # e1 = cv2.getTickCount()
            count += 1
            # frame = cv2.putText(
            #     frame,
            #     f"FPS: {fps}",
            #     (10, 30),
            #     cv2.FONT_HERSHEY_PLAIN,
            #     1,
            #     (0, 255, 0),
            #     2,
            # )
            # frame = cv2.flip(frame, -1)  # 상하 좌우 대
            if width > height:
                width_tmp = 720
                height_tmp = 480
            else:
                width_tmp = 480
                height_tmp = 720
            frame = cv2.resize(
                frame, dsize=(width_tmp, height_tmp), interpolation=cv2.INTER_AREA
            )
            # frame = cv2.resize(
            # frame, dsize=(width/2, height/2), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA
            # )
            point = get_img_coord(frame)
            for p in point:
                if p[2] > 0.01:
                    cv2.circle(frame, (int(p[0]), int(p[1])), 2, (0, 0, 255))
            for link in link_parts:
                # print(link)
                p1 = body_parts.index(link[0])
                p2 = body_parts.index(link[1])
                if point[p1][2] > 0.01 and point[p2][2] > 0.01:
                    cv2.line(frame, (int(point[p1][0]), int(point[p1][1])), (int(point[p2][0]), int(point[p2][1])), (0, 255, 0), 2)
            if count % flag == 0:
                print(count)
                label.append(angle.out(point, body_parts, label_parts))

                if len(label) > window_size:
                    label.pop(0)

                if len(label) == window_size:
                    # label_ = (label)
                    # label_ = label_.reshape(-1, label.shape[0] * label.shape[1])
                    # print(f"label: {np.shape([label])}")
                    label_ = np.array([label]).reshape(-1)
                    pred = model.predict([label_])
                    # print(pred)
                    try:
                        la_ = encoder.inverse_transform(pred)
                        result = la_
                        print(f"pred: {la_}")
                    except:
                        print("분류하지 못함")
                        # result = "분류하지 못함"
                # print(f"count: {count}, label: {np.shape(label)}")

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            # e2 = cv2.getTickCount()
            # time = (e2 - e1) / cv2.getTickFrequency()
            # print(f"time: {time}")
            # cv2.imshow(f"video", frame)
            cv2.putText(
                frame,
                f"result: {result}",
                (10, 30),
                cv2.FONT_HERSHEY_PLAIN,
                1,
                (0, 255, 0),
                2,
            )
            if video_save:
                out_video.append(frame)
            ret, frame = cap.read()
    else:
        print("Error")

    cap.release()
    cv2.destroyAllWindows()

    if video_save:
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        video_name = video.split("/")[-1].split(".")[0]
        out = cv2.VideoWriter(
            f"out/{video_name}_predic.mp4", fourcc, fps, (width_tmp, height_tmp)
        )

        for i in range(len(out_video)):
            out.write(out_video[i])
        out.release()


predict(video_save=True)

width: 720, height: 1280, fps: 15.0
2
4
6
8
10
12
14
16
18
20
분류하지 못함
22
분류하지 못함
24
분류하지 못함
26
분류하지 못함
28
분류하지 못함
30
분류하지 못함
32
분류하지 못함
34
분류하지 못함
36
분류하지 못함
38
분류하지 못함
40
분류하지 못함
42
분류하지 못함
44
분류하지 못함
46
분류하지 못함
48
분류하지 못함
50
분류하지 못함
52
분류하지 못함
54
pred: [['WALKRUN']]
56
분류하지 못함
58
pred: [['BODYLOWER']]
60
분류하지 못함
62
분류하지 못함
64
분류하지 못함
66
분류하지 못함
68
분류하지 못함
70
분류하지 못함
72
분류하지 못함
74
분류하지 못함
76
분류하지 못함
78
분류하지 못함
80
분류하지 못함
82
분류하지 못함
84
pred: [['TAILING']]
86
분류하지 못함
88
분류하지 못함
90
분류하지 못함
92
pred: [['BODYLOWER']]
94
분류하지 못함
96
분류하지 못함
98
분류하지 못함
100
분류하지 못함
102
분류하지 못함
104
분류하지 못함
106
분류하지 못함
108
분류하지 못함
110
분류하지 못함
112
분류하지 못함
114
분류하지 못함


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
